In [9]:
import os
external_drive_path = "/Volumes/Crucial X9 Pro For Mac/TP_Vision_Final/train_allmetadata_json"
# Check if the drive is accessible
if os.path.exists(external_drive_path):
    print("External drive found!")
else:
    print("External drive not found. Check the path.")

External drive found!


In [10]:
import pandas as pd
import numpy as np

In [11]:
#set random_seed
np.random.seed(42)

### subsample

In [12]:
#DEFINE CUANTAS FOTOS AGARRAR
n_photos =80000

In [13]:
file_paths = pd.read_csv(external_drive_path+'/train_img_filepath.txt', delim_whitespace=True, header=None)
targuets = pd.read_csv(external_drive_path+'/train_label.txt', delim_whitespace=True, header=None)

#50k random number withour repetition 

random_numbers = range(0, n_photos)
print(len(random_numbers))

paths = file_paths.iloc[random_numbers].values
targuets = targuets.iloc[random_numbers].values

/var/folders/7f/w8t6fsgd78gfxd_lgh6918b00000gn/T/ipykernel_7234/1164682750.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  file_paths = pd.read_csv(external_drive_path+'/train_img_filepath.txt', delim_whitespace=True, header=None)


80000


/var/folders/7f/w8t6fsgd78gfxd_lgh6918b00000gn/T/ipykernel_7234/1164682750.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  targuets = pd.read_csv(external_drive_path+'/train_label.txt', delim_whitespace=True, header=None)


In [14]:
uids = []
pids = []
for i in range(len(paths)):
    uid = paths[i][0].split('/')[1]
    pid = paths[i][0].split('/')[2].split('.')[0]
    uids.append(uid)
    pids.append(pid)

uids = [str(uid) for uid in uids]
pids = [str(pid) for pid in pids]
targuets = [float(targuet) for targuet in targuets]

#create a datgrame from the lists

dic = {
    'Uid': uids,
    'Pid': pids,
    'targuet': targuets
}
df = pd.DataFrame(dic)
df = pd.DataFrame(dic)

print (dic['Uid'][0], dic['Pid'][0], dic['targuet'][0])

/var/folders/7f/w8t6fsgd78gfxd_lgh6918b00000gn/T/ipykernel_7234/2607862423.py:11: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  targuets = [float(targuet) for targuet in targuets]


59@N75 775 11.18


In [15]:
#read json file and filter data 
category = pd.read_json(external_drive_path+'/train_category.json')
temporal = pd.read_json(external_drive_path+'/train_temporalspatial_information.json')
user_data = pd.read_json(external_drive_path+'/train_user_data.json')



category = category[category['Uid'].astype(str).isin(dic['Uid'])]
category = category[category['Pid'].astype(str).isin(dic['Pid'])]
category['Uid'] = category['Uid'].astype(str)
category['Pid'] = category['Pid'].astype(str)

temporal = temporal[temporal['Uid'].astype(str).isin(dic['Uid'])]
temporal = temporal[temporal['Pid'].astype(str).isin(dic['Pid'])]
temporal.drop(['Longitude', 'Geoaccuracy', 'Latitude'], axis=1, inplace=True)
temporal['Uid'] = temporal['Uid'].astype(str)
temporal['Pid'] = temporal['Pid'].astype(str)

user_data = user_data[user_data['Uid'].astype(str).isin(dic['Uid'])]
user_data = user_data[user_data['Pid'].astype(str).isin(dic['Pid'])]
user_data.drop(['photo_firstdatetaken', 'timezone_id', 'user_description', 'location_description'], axis=1, inplace=True)
user_data['Uid'] = user_data['Uid'].astype(str)
user_data['Pid'] = user_data['Pid'].astype(str)


print("el largo de user_data de es", len(user_data))
print("el largo de categorias de es", len(category))
print("el largo de temporales de es", len(temporal))

el largo de user_data de es 80000
el largo de categorias de es 80000
el largo de temporales de es 80000


In [16]:
#merge data with targuet in order
#create dataframe from the lists with the data

df = pd.merge(df, category, how='inner', on=['Uid', 'Pid'])
df = pd.merge(df, temporal, how='inner', on=['Uid', 'Pid'])
df = pd.merge(df, user_data, how='inner', on=['Uid', 'Pid'])

In [17]:
print(df.head())
print(print("el largo de todo es de es", len(df)))
print(df.columns)
print("tiene cantidad de columnas", len(df.columns))

        Uid   Pid  targuet              Category  Concept Subcategory  \
0    59@N75   775    11.18               Fashion     glam     Fashion   
1     1@N18  1075    15.15  Travel&Active&Sports     fifa      Soccer   
2   351@N64  4890    10.99         Entertainment   cinema      Movies   
3     6@N59  6568     8.63  Holiday&Celebrations      old    Birthday   
4  1617@N40  7079    11.16                  Food  thirsty      Drinks   

     Postdate photo_firstdate  photo_count  ispro timezone_offset  
0  1446016778            None       1429.0    1.0            None  
1  1454983379            None      43108.0    0.0          -03:00  
2  1433118604            None       1035.0    1.0          +01:00  
3  1451577600            None      83322.0    1.0          +01:00  
4  1425744438            None       5958.0    1.0          -02:00  
el largo de todo es de es 80000
None
Index(['Uid', 'Pid', 'targuet', 'Category', 'Concept', 'Subcategory',
       'Postdate', 'photo_firstdate', 'photo_c

### preprocess

In [18]:
#one hot encode category
one_hot_category = pd.get_dummies(df['Category'])
one_hot_subcategory = pd.get_dummies(df['Subcategory'])
one_hot_concept = pd.get_dummies(df['Concept'])

df.drop(['Category', 'Subcategory', 'Concept'], axis=1, inplace=True)

df = pd.concat([df, one_hot_category, one_hot_subcategory, one_hot_concept], axis=1)

In [19]:
def one_hot_encode_date_features(df, date_column):
    """
    Realiza one-hot encoding de los meses y días de la semana a partir de un timestamp Unix.
    
    Args:
        df (pd.DataFrame): DataFrame que contiene la columna de fechas en formato Unix.
        date_column (str): Nombre de la columna que contiene las fechas en formato Unix.
        
    Returns:
        pd.DataFrame: DataFrame original con las columnas codificadas.
    """
    # Convertir la columna de Unix timestamps a datetime
    df['datetime'] = pd.to_datetime(df[date_column], unit='s')
    
    # Extraer el mes y el día de la semana
    df['month'] = df['datetime'].dt.month  # Mes (1 a 12)
    df['weekday'] = df['datetime'].dt.dayofweek  # Día de la semana (0=lunes, 6=domingo)
    
    # Realizar one-hot encoding
    one_hot_month = pd.get_dummies(df['month'], prefix='month')
    one_hot_weekday = pd.get_dummies(df['weekday'], prefix='weekday')
    
    # Concatenar las columnas codificadas con el DataFrame original
    df = pd.concat([df, one_hot_month, one_hot_weekday], axis=1)
    
    # Eliminar columnas temporales
    df = df.drop(['datetime', 'month', 'weekday'], axis=1)
    
    return df

df = one_hot_encode_date_features(df, 'Postdate')

In [20]:
print(df.head())
print(print("el largo de todo es de es", len(df)))
print(df.columns)
print("tiene cantidad de columnas", len(df.columns))

        Uid   Pid  targuet    Postdate photo_firstdate  photo_count  ispro  \
0    59@N75   775    11.18  1446016778            None       1429.0    1.0   
1     1@N18  1075    15.15  1454983379            None      43108.0    0.0   
2   351@N64  4890    10.99  1433118604            None       1035.0    1.0   
3     6@N59  6568     8.63  1451577600            None      83322.0    1.0   
4  1617@N40  7079    11.16  1425744438            None       5958.0    1.0   

  timezone_offset  Animal  Electronics  ...  month_10  month_11  month_12  \
0            None   False        False  ...      True     False     False   
1          -03:00   False        False  ...     False     False     False   
2          +01:00   False        False  ...     False     False     False   
3          +01:00   False        False  ...     False     False      True   
4          -02:00   False        False  ...     False     False     False   

   weekday_0  weekday_1  weekday_2  weekday_3  weekday_4  weekday_5 

In [21]:
#save the data as csv
df.to_csv('../data/metadata/all_metadata.csv', index=False)